In [ ]:
pwd

In [ ]:
import os 
os.chdir("./..")

In [ ]:
pwd

In [ ]:
import os
import urllib.request as request
import zipfile
import tarfile
import time
import concurrent.futures
import smtplib
from base64 import b64encode
from cnnClassifier import logger
from cnnClassifier.constants import *
from cnnClassifier.utils.common_functions import get_size, read_yaml, create_directories
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    auth_method: str
    api_key: str = None
    username: str = None
    password: str = None
    oauth_token: str = None

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    # def get_data_ingestion_config(self) -> DataIngestionConfig:
    #     config = self.config.data_ingestion
    #     create_directories([config.root_dir])
    #     data_ingestion_config = DataIngestionConfig(
    #         root_dir = config.root_dir,
    #         source_URL = config.source_URL,
    #         local_data_file = config.local_data_file,
    #         unzip_dir = config.unzip_dir
    #     ) 
    #     return data_ingestion_config
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        return DataIngestionConfig(**config)
        
class DataIngestion:
    MAX_RETRIES = 3

    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def authenticate(self):
        headers = {}
        if self.config.auth_method == "api_key":
            headers["Authorization"] = f"Bearer {self.config.api_key}"
        elif self.config.auth_method == "basic_auth":
            credentials = b64encode(f"{self.config.username}:{self.config.password}".encode()).decode()
            headers["Authorization"] = f"Basic {credentials}"
        elif self.config.auth_method == "oauth":
            headers["Authorization"] = f"Bearer {self.config.oauth_token}"
        else:
            raise ValueError("Unsupported authentication method")
        return headers
        
    def download_file(self):
        headers = self.authenticate()
        req = request.Request(self.config.source_URL, headers=headers)
        retries = 0
        while retries < self.MAX_RETRIES:
            try:
                with request.urlopen(req) as response, open(self.config.local_data_file, 'wb') as out_file:
                    out_file.write(response.read())
                break
            except Exception as e:
                retries += 1
                logger.error(f"Error downloading file (Attempt {retries}): {e}")
                time.sleep(5)
        if retries == self.MAX_RETRIES:
            self.notify_failure("Download failed after maximum retries.")

    def extract_file(self):
        try:
            if self.config.local_data_file.endswith('.zip'):
                with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
                    zip_ref.extractall(self.config.unzip_dir)
            elif self.config.local_data_file.endswith('.tar.gz'):
                with tarfile.open(self.config.local_data_file, "r:gz") as tar_ref:
                    tar_ref.extractall(self.config.unzip_dir)
            else:
                logger.warning(f"Unsupported file format: {self.config.local_data_file}")
        except Exception as e:
            logger.error(f"Error extracting file: {e}")
            raise

    def validate_data(self):
        if os.path.getsize(self.config.local_data_file) == 0:
            raise ValueError("Downloaded file is empty")

    def notify_failure(self, message):
        # Email Notification
        with smtplib.SMTP('smtp.example.com', 587) as server:
            server.login("mekki.amine01@gmail.com", "Amine-1963")
            server.sendmail("mekki.amine01@gmail.com", "amineraps@gmail.com", message)

